# Variography

<!-- SUMMARY: Calculate variograms on one or several variables of a Db, for any space dimension. Fit a Model (automatic procedure)  -->

<!-- CATEGORY: Courses -->

In this preamble, we load the **gstlearn** library.

## Preamble

In [ ]:
import gstlearn as gl
import gstlearn.plot as gp
import gstlearn.document as gdoc
import matplotlib.pyplot as plt
import numpy as np
import os
from IPython.display import Markdown

gdoc.setNoScroll()

Then the necessary data set is downloaded and named **dat**: the target variable is **January_temp**

In [2]:
temp_nf = gdoc.loadData("Scotland", "Scotland_Temperatures.NF")
dat = gl.Db.createFromNF(temp_nf)

## Variogram Cloud

In [ ]:
Markdown(gdoc.loadDoc("Variogram_Cloud.md"))

In [ ]:
varioParamOmni = gl.VarioParam.createOmniDirection(100)
grid_cloud = gl.db_vcloud(dat, varioParamOmni)
grid_cloud.display()

In [ ]:
ax = grid_cloud.plot("Cloud.January*")
plt.gca().set_aspect('100')
ax.decoration(title="Variogram Cloud")

## Experimental (isotropic) variograms

In [ ]:
Markdown(gdoc.loadDoc("Experimental_Variogram.md"))

In [ ]:
varioParamOmni = gl.VarioParam.createOmniDirection(nlag=40, dlag=10, toldis=0.1)

dat.setLocator("January_temp",gl.ELoc.Z)
varioexp = gl.Vario(varioParamOmni)
err = varioexp.compute(dat)

We now print the contents of the newly created experimental variogram. The $40$ experimental variogram values are displayed (Columun `Value`), together with the number $\vert \widehat N(h)\vert$ of pairs used to compute the value (Columun `Npairs`) and the average distance between the points forming these pairs (Column `Distance`).

In [ ]:
varioexp

We now plot the experimental variogram. In the resulting figure, the experimental variogram is plotted in blue, and the dashed blacked line corresponds to the value of the variance of the data.

In [ ]:
gp.setDefault(dims=[6,6])
gp.varmod(varioexp)
plt.show()

We can also adapt the size of experimental variogram points in the plot so that it is proportional to the number of pairs of points used to compute the value. 

In [ ]:
ax = gp.varmod(varioexp,showPairs=True)

## Automatic Model Fitting

Fitting a variogram model on an experimental variogram is done in two steps. First, we create `Model` object. These objects aim at containing all the necessary information about the covariance structure of a random field. In particular, it is assumed that this covariance structure is a superposition of basic elementary covariance structures: the `Model` objects then contains the covariance types and parameters of each one of these basic covariance structures.

In our case, we wish to build our `Model` object from an experimental variogram, meaning that we want to find a composition of basic covariance structures which would result in a variogram "close" to the experimental variogram that we computed from the data. This is done by calling the method `fit` of the `Model` object, while providing it with the experimental variogram.

In the next example, we create a `Model` object, that we fit on the experimental variogram the we computed earlier. We then plot both the experimental variogram and the variogram model resulting from the fitting using the `plot.varmod` function. In the figure we obtain, In the figure above, the dashed blue line corresponds to the experimental variogram, and the solid blue line corresponds to the fitted variogram model.

In [ ]:
fitmod = gl.Model()
err = fitmod.fit(varioexp)

In [ ]:
gp.varmod(varioexp, fitmod)
plt.show()

We now print the content of our newly created model. As we can see, only one basic covariance structure is used to define the model (namely, a Spherical covariance function whose range and sill are printed).

In [ ]:
fitmod

### Model Fitting with pre-defined basic structures

It is also possible to guide the model fitting by proposing a list of basic covariance structures from which the model is to be built. The list of available basic covariance structures is obtained by running the following command:

In [ ]:
gl.ECov.printAll()

In practice, we start by creating a list of basic structures using the `ECov_fromKeys` function which we supply with a vector containing the names of the basic structures we would like to see in the model. To fit the model, we then once again call the `fit` method and supply it with both the experimental variogram and the newly created list of basic structures (argument `types`). Then the fitting procedures tries find the composition of models from the supplied list that best fits the experimental variogram. 

Note that by default, the fitting algorithm tries to be parsimonious and can therefore "drop" some of the structures that we supply if it deems that a model with less structures provides a better fit. To force the fitting algorithm to keep all the structures from the list, we  simply need to add the argument `optvar=Option_VarioFit(TRUE)` to the `fit` method.

In the next example, we once again define a model by fitting it on our experimental variogram. But this time, we specify that we want the resulting model to be a composition of basic structures restricted to these choices: a Nugget effect, a Cubic covariance and a Spherical covariance. 

In [ ]:
types = [gl.ECov.NUGGET, gl.ECov.CUBIC, gl.ECov.SPHERICAL]
err = fitmod.fit(varioexp, types=types)

In [ ]:
ax = gp.varmod(varioexp, fitmod)

When printing the contents of the model, we now notice that it consists of a superposition of a Cubic covariance and a Spherical covariance, as intended. Note that the Nugget effect does not appear (it has been dropped by the fitting algorithm).

In [ ]:
fitmod

### Model Fitting with constraints

It is possible to impose (in)equality constraints on the covariance parameters of the basic structures used in the model fitting procedure. This is done by creating a `Constraints` object that is used to specify the constraints we wish to impose on the parameters of the different basic structures composing the model. To add a constraint to the object, we can use the method `addItemFromParamId`, which takes as arguments the type of parameter for which the constraint applies (given as an `EConsElem` object: run `EConsElem_printAll()` for the list of available options), the index of the basic structure for which the constraint applies (argument `icov`), the type of constraint we wish to apply (argument `type`, given as an `EConsType` object: run `EConsType_printAll()` for the list of available options) and finally the numerical value (argument `value`) defining the constraint.

In the next example, we start from a list of three basic structures (a Nugget effect, a Cubic covariance and a Spherical covariance), and create a `Constraints` object conatining two constrainits. The first one applies to the basic structure of index $1$ (the cubic structure), and sets an upper-bound of $20$ for its range. The second one also applies to the basic structure of index $1$ (the cubic structure), and sets an lower-bound of $0.03$ for its sill. Finally, the `fit` method is called to fit the model on the experimental variogram. Note that we also added the option `optvar=Option_VarioFit(TRUE)` to force the fitting algorithm to keep the three basic structures that we supplied.

In [ ]:
types = gl.ECov.fromKeys(["NUGGET","CUBIC","SPHERICAL"])
constraints = gl.Constraints()
err = constraints.addItemFromParamId(gl.EConsElem.RANGE,icov=1,type=gl.EConsType.UPPER,value=20.)
err = constraints.addItemFromParamId(gl.EConsElem.SILL,icov=1,type=gl.EConsType.LOWER,value=0.03)
err = fitmod.fit(varioexp, types, constraints, optvar=gl.Option_VarioFit(True))

In [ ]:
ax = gp.varmod(varioexp, fitmod)

When printing the content of the fitted model, we see that the constraints are indeed satisfied (and that the three basic structures are present).

In [ ]:
fitmod

In the following example, we now apply equality constraints to the parameters. The first one applies to the basic structure of index $1$ (the cubic structure), and sets its range to the value $1000$. The second one also applies to the basic structure of index $1$ (the cubic structure), and sets its sill to the value $0.4$. 

In [ ]:
constraints = gl.Constraints()
err = constraints.addItemFromParamId(gl.EConsElem.RANGE,icov=1,type=gl.EConsType.EQUAL,value=1000.)
err = constraints.addItemFromParamId(gl.EConsElem.SILL,icov=1,type=gl.EConsType.EQUAL,value=0.4)
err = fitmod.fit(varioexp, types, constraints, gl.Option_VarioFit(True))

In [ ]:
ax = gp.varmod(varioexp, fitmod)

When printing the content of the fitted model, we see that the constraints are once again satisfied (and that the three basic structures are present).

In [ ]:
fitmod

## Directional Variograms

In [ ]:
Markdown(gdoc.loadDoc("Directional_Variogram.md"))

In [ ]:
varioParamMulti = gl.VarioParam.createMultiple(ndir=4, nlag=15, dlag=15.)
vario_4dir = gl.Vario(varioParamMulti)
err = vario_4dir.compute(dat)

In [ ]:
ax = gp.varmod(vario_4dir, flagLegend=True)

Then, fitting a model onto the resulting experimental variogram is done using the same commands as in the isotropic case.

In [ ]:
model_4dir = gl.Model()
err = model_4dir.fit(vario_4dir,types=types)

In [ ]:
ax = gp.varmod(vario_4dir, model_4dir)

## Variogram Maps

In [ ]:
Markdown(gdoc.loadDoc("Variogram_Map.md"))

In [ ]:
grid_vmap = gl.db_vmap(dat)

In [ ]:
fig, ax = plt.subplots(1,2,figsize=[14,10])
fig.tight_layout(pad=5.0)
ax[0].raster(grid_vmap, flagLegend=True)
ax[1].raster(grid_vmap, name="*Nb", flagLegend=True)
plt.show()

It is then possible to fit a model directly on the experimental variogram map. This if done with the method `fitFromVMap` from the `Model` class. This method is called in the same way as the `fit` method considered up until now (the experimental variograms being now replaced by the experimental variogram map).

In [ ]:
modelVM = gl.Model()
err = modelVM.fitFromVMap(grid_vmap, types=types)
modelVM

It is then possible to plot the variogram map resulting from the fitted model. To do so, we start by evaluating the fitted variogram model on the the experimental variogram map grid. This is done using the function `buildVmapOnDbGrid` which we supply with both the experimental variogram map and the fitted model. This function adds a additional variable to the `Db` containing the experimental variogram map corresponding to the evaluations of the variogram model.

In [ ]:
err = modelVM.buildVmapOnDbGrid(grid_vmap)

In [ ]:
ax = gp.raster(grid_vmap)

Finally, we plot together the experimental directional variograms and the model obtained from fitting the variogram map.

In [ ]:
ax = gp.varmod(vario_4dir, modelVM)